# Ejercicio de programación Regresión Lineal Multiple

In [44]:
# utilizado para manejos de directorios y rutas
import os

# Computacion vectorial y cientifica para python
import numpy as np

# Librerias para graficación (trazado de gráficos)
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D  # Necesario para graficar superficies 3D
import pandas as pd
import csv
# llama a matplotlib a embeber graficas dentro de los cuadernillos
%matplotlib inline

In [103]:
data = pd.read_csv("./DataSet/survey.csv", delimiter=',')
data.describe()

,age,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,q17,q18,q19,q20,q21,q22,q23,q24,q25,q26
count,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,...,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000
mean,24.863014,8.013699,7.863014,7.821918,7.410959,7.506849,7.643836,7.630137,7.602740,7.068493,...,8.068493,7.452055,7.465753,7.410959,7.589041,7.808219,7.561644,7.479452,7.561644,7.328767
std,12.484785,2.195280,1.953025,2.231980,2.040123,1.764805,2.002567,1.961482,1.905691,2.281135,...,2.070485,2.041521,2.297838,2.178402,2.408524,2.189987,2.471989,2.298004,2.279049,2.333659
min,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,19.000000,7.000000,7.000000,7.000000,6.000000,7.000000,7.000000,7.000000,7.000000,6.000000,...,7.000000,6.000000,6.000000,6.000000,7.000000,7.000000,6.000000,7.000000,7.000000,6.000000
50%,20.000000,9.000000,8.000000,9.000000,8.000000,8.000000,8.000000,8.000000,8.000000,7.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
75%,29.000000,10.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,10.000000,9.000000,9.000000,9.000000,9.000000,9.000000,10.000000,9.000000,9.000000,9.000000
max,55.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [76]:
#data = pd.read_csv("./DataSet/survey.csv", delimiter=';', skiprows=1)
#data = np.loadtxt('./DataSet/survey.csv', delimiter=',', skiprows=1,)
data = pd.read_csv("./DataSet/survey.csv", delimiter=',')
data1 = pd.DataFrame(data)
X = data1[["q1","q4","q23"]]
y = data["q26"]
print("x\n", X, "y\n",y)
m = y.size

x
     q1  q4  q23
0    9   6    5
1   10   9   10
2   10  10    9
3    5   5   10
4    9  10   10
..  ..  ..  ...
68   9   9    5
69   8   9    6
70  10   6    1
71  10  10   10
72   1   2   10

[73 rows x 3 columns] y
 0      3
1      8
2      8
3      6
4     10
      ..
68     5
69     6
70     1
71    10
72    10
Name: q26, Length: 73, dtype: int64


In [77]:
def graficarDatos(x, y):
    fig = pyplot.figure()
    pyplot.plot(x, y, 'ro')
    pyplot.xlabel('y')
    pyplot.ylabel(' x')

## 2 Regresión lineal con multiples variables

Se implementa la regresion lineal multivariable para predecir el precio de las casas. El archivo `Datasets/ex1data2.txt` contiene un conjunto de entrenamiento de precios de casas en Portland, Oregon. La primera columna es el tamaño de la casa en metros cuadrados, la segunda columna es el numero de cuartos, y la tercera columna es el precio de la casa. 

<a id="section4"></a>
### 2.1 Normalización de caracteristicas

Al visualizar los datos se puede observar que las caracteristicas tienen diferentes magnitudes, por lo cual se debe transformar cada valor en una escala de valores similares, esto con el fin de que el descenso por el gradiente pueda converger mas rapidamente.

La desviación estándar es una forma de medir cuánta variación hay en el rango de valores de una característica en particular (la mayoría de los puntos caeran en un rango de ± 2 en relación a la desviaciones estándar de la media); esta es una alternativa a tomar el rango de valores (max-min). En `numpy`, se puede usar la función `std` para calcular la desviacion estandar. 

Por ejemplo, la caracteristica`X[:, 0]` contiene todos los valores de $x_1$ (tamaño de las casas) en el conjunto de entrenamiento, entonces `np.std(X[:, 0])` calcula la desviacion estandar de los tamaños de las casas.
En el momento en que se llama a la función `featureNormalize`, la columna adicional de unos correspondiente a $ x_0 = 1 $ aún no se ha agregado a $ X $. 

<div class="alert alert-block alert-warning">
**Notas para la implementación:** Cuando se normalize una caracteristica, es importante almacenar los valores usados para la normalización - el valor de la media y el valor de la desviación estandar usado para los calculos. Despues de aprender los parametros del modelo, se deseara predecir los precios de casas que no se han visto antes. Dado un nuevo valor de x (area del living room y el numero de dormitorios), primero se debe normalizar x usando la media y la desviacion estandar que se empleo anteriormente en el conjunto de entrenamiento para entrenar el modelo.
</div>
<a id="featureNormalize"></a>

In [79]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma
    
    return X_norm, mu, sigma

In [80]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

#print(X)
print('Media calculada:', mu)
print('Desviación estandar calculada:', sigma)
#print(X_norm)

Media calculada: q1     8.013699
q4     7.410959
q23    7.561644
dtype: float64
Desviación estandar calculada: q1     2.180192
q4     2.026101
q23    2.454999
dtype: float64


Despues de `featureNormalize` la funcion es provada, se añade el temino de interseccion a `X_norm`:

In [81]:
# Añade el termino de interseccion a X
# (Columna de unos para X0)
X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)

In [82]:
print(X)

[[ 1.          0.45239198 -0.69639119 -1.04343971]
 [ 1.          0.91106719  0.78428522  0.99322068]
 [ 1.          0.91106719  1.27784403  0.58588861]
 [ 1.         -1.38230884 -1.18994999  0.99322068]
 [ 1.          0.45239198  1.27784403  0.99322068]
 [ 1.          0.91106719  0.78428522  0.99322068]
 [ 1.         -0.00628322 -0.69639119 -1.04343971]
 [ 1.          0.45239198 -0.69639119 -2.26543594]
 [ 1.         -0.00628322  0.78428522  0.58588861]
 [ 1.         -0.46495843 -0.20283239 -0.22877555]
 [ 1.          0.45239198  0.29072642  0.58588861]
 [ 1.          0.91106719  1.27784403  0.99322068]
 [ 1.          0.91106719  0.29072642  0.99322068]
 [ 1.          0.91106719 -0.69639119  0.99322068]
 [ 1.          0.91106719  0.29072642  0.99322068]
 [ 1.          0.91106719  1.27784403  0.58588861]
 [ 1.         -1.84098405 -1.6835088  -1.45077179]
 [ 1.          0.45239198 -0.20283239 -0.22877555]
 [ 1.         -0.00628322  0.78428522 -0.22877555]
 [ 1.         -0.46495843  0.29

<a id="section5"></a>
### 2.2 Descenso por el gradiente

En el ejemplo anterior se implemento el descenso por el gradiente para un problema de regresion univariable. La unica diferencia es que ahora existe una caracteristica adicional en la matriz $X$. La función de hipótesis y la regla de actualización del descenso del gradiente por lotes permanecen sin cambios.

La implementacion de las funciones `computeCostMulti` y `gradientDescentMulti` son similares a la funcion de costo y función de descenso por el gradiente de la regresión lineal multiple es similar al de la regresion lineal multivariable. Es importante garantizar que el codigo soporte cualquier numero de caracteristicas y esten bien vectorizadas.

Se puede utilizar `shape`, propiedad de los arrays `numpy`, para identificar cuantas caracteristicas estan consideradas en el dataset.

<div class="alert alert-block alert-warning">
**Nota de implementación:** En el caso de multivariables, la función de costo puede se escrita considerando la forma vectorizada de la siguiente manera:

$$ J(\theta) = \frac{1}{2m}(X\theta - \vec{y})^T(X\theta - \vec{y}) $$

donde:

$$ X = \begin{pmatrix}
          - (x^{(1)})^T - \\
          - (x^{(2)})^T - \\
          \vdots \\
          - (x^{(m)})^T - \\ \\
        \end{pmatrix} \qquad \mathbf{y} = \begin{bmatrix} y^{(1)} \\ y^{(2)} \\ \vdots \\ y^{(m)} \\\end{bmatrix}$$

La version vectorizada es eficiente cuando se trabaja con herramientas de calculo numericos computacional como `numpy`. 
</div>

<a id="computeCostMulti"></a>

In [83]:
def computeCostMulti(X, y, theta):
    # Inicializa algunos valores utiles
    m = y.shape[0] # numero de ejemplos de entrenamiento
    
    J = 0
    
    h = np.dot(X, theta)
    
    J = (1/(2 * m)) * np.sum(np.square(np.dot(X, theta) - y))
    
    return J


In [84]:
def gradientDescentMulti(X, y, theta, alpha, num_iters):
    
    # Inicializa algunos valores 
    m = y.shape[0] # numero de ejemplos de entrenamiento
    
    # realiza una copia de theta, el cual será acutalizada por el descenso por el gradiente
    theta = theta.copy()
    
    J_history = []
    
    for i in range(num_iters):
        theta = theta - (alpha / m) * (np.dot(X, theta) - y).dot(X)
        J_history.append(computeCostMulti(X, y, theta))
    
    return theta, J_history

#### 3.2.1 Seleccionando coheficientes de aprendizaje


theta calculado por el descenso por el gradiente: [7.32876712 0.14542051 0.05668689 1.73451104]
(usando el descenso por el gradiente)
 El sitio web es recomendable a: 7 amigos


In [93]:
# Elegir algun valor para alpha (probar varias alternativas)
alpha = 0.003 # alpha = 0.003
num_iters = 10000

# inicializa theta y ejecuta el descenso por el gradiente
theta = np.zeros(4)
theta, J_history = gradientDescentMulti(X, y, theta, alpha, num_iters)

# Grafica la convergencia del costo
pyplot.plot(np.arange(len(J_history)), J_history, lw=2)
pyplot.xlabel('Numero de iteraciones')
pyplot.ylabel('Costo J')

# Muestra los resultados del descenso por el gradiente
print('theta calculado por el descenso por el gradiente: {:s}'.format(str(theta)))

# Estimar a cuantas personas cuantas amigos personas se pueden recomendar el sitio web
X_array = [1, 3, 1, 8]
X_array[1:4] = (X_array[1:4] - mu) / sigma
predit = np.dot(X_array, theta)   # Se debe cambiar esto

print('(usando el descenso por el gradiente)\n El sitio web es recomendable a: {:.0f}'.format(predit), 'amigos')

In [94]:
X_array = [1, 10, 3, 4]
X_array[1:4] = (X_array[1:4] - mu) / sigma

In [95]:
X_array[1:4]

[0.911067191137229, -2.177067599204224, -1.4507717862449523]

<a id="section7"></a>
### 2.3 Ecuacion de la Normal

Una manera de calcular rapidamente el modelo de una regresion lineal es:

$$ \theta = \left( X^T X\right)^{-1} X^T\vec{y}$$

Utilizando esta formula no requiere que se escale ninguna caracteristica, y se obtendra una solucion exacta con un solo calculo: no hay “bucles de convergencia” como en el descenso por el gradiente. 

Primero se recargan los datos para garantizar que las variables no esten modificadas. Recordar que no es necesario escalar las caracteristicas, se debe agregar la columna de unos a la matriz $X$ para tener el termino de intersección($\theta_0$). 

In [98]:
# Cargar datos
data = pd.read_csv("./DataSet/survey.csv")
data1 = pd.DataFrame(data)
X = data[["q14","q23"]]
y = data["age"]
m = y.size
X = np.concatenate([np.ones((m, 1)), X], axis=1)
print(X)

[[ 1.  4.  5.]
 [ 1. 10. 10.]
 [ 1.  9.  9.]
 [ 1.  9. 10.]
 [ 1.  9. 10.]
 [ 1.  4. 10.]
 [ 1.  6.  5.]
 [ 1. 10.  2.]
 [ 1.  8.  9.]
 [ 1.  8.  7.]
 [ 1.  9.  9.]
 [ 1. 10. 10.]
 [ 1.  9. 10.]
 [ 1.  9. 10.]
 [ 1. 10. 10.]
 [ 1.  8.  9.]
 [ 1.  4.  4.]
 [ 1.  6.  7.]
 [ 1.  9.  7.]
 [ 1.  8.  7.]
 [ 1.  9.  8.]
 [ 1. 10.  8.]
 [ 1.  8.  9.]
 [ 1.  8.  9.]
 [ 1.  5.  1.]
 [ 1.  7. 10.]
 [ 1.  9.  7.]
 [ 1.  7.  8.]
 [ 1.  9.  8.]
 [ 1.  7.  6.]
 [ 1.  5.  6.]
 [ 1.  7.  9.]
 [ 1.  7.  7.]
 [ 1.  9. 10.]
 [ 1.  7.  8.]
 [ 1.  5.  3.]
 [ 1.  7.  6.]
 [ 1. 10.  7.]
 [ 1.  7. 10.]
 [ 1.  7.  7.]
 [ 1.  7.  1.]
 [ 1. 10. 10.]
 [ 1.  9.  9.]
 [ 1.  9. 10.]
 [ 1. 10. 10.]
 [ 1.  6.  7.]
 [ 1.  8.  8.]
 [ 1.  6.  7.]
 [ 1.  6.  5.]
 [ 1.  9.  8.]
 [ 1.  1.  2.]
 [ 1.  7.  5.]
 [ 1.  7.  9.]
 [ 1.  5.  6.]
 [ 1.  9.  9.]
 [ 1. 10. 10.]
 [ 1.  9.  8.]
 [ 1.  7.  6.]
 [ 1.  9. 10.]
 [ 1.  8.  8.]
 [ 1.  9. 10.]
 [ 1.  5.  5.]
 [ 1.  7.  9.]
 [ 1.  7.  9.]
 [ 1. 10. 10.]
 [ 1. 10.  5.]
 [ 1.  8. 

In [99]:
def normalEqn(X, y):
  
    theta = np.zeros(X.shape[1])
    
    theta = np.dot(np.dot(np.linalg.inv(np.dot(X.T,X)),X.T),y)
    
    return theta

In [102]:
# Calcula los parametros con la ecuación de la normal
theta = normalEqn(X, y);

# Muestra los resultados optenidos a partir de la aplicación de la ecuación de la normal
print('Theta calculado a partir de la ecuación de la normal: {:s}'.format(str(theta)));

# Estimar el precio para una casa de superficie de 1650 sq-ft y tres dormitorios

X_array = [1, 8, 10]
predit = np.dot(X_array, theta) 

print('Edad predecido en la satisfación del sitio web : {:.0f}'.format(predit), "anos")

Theta calculado a partir de la ecuación de la normal: [36.27895976 -1.6143258   0.13092882]
Edad predecido en la satisfación del sitio web : 25 anos
